## CV10 - Class Activation Map
### CAM, Grad-CAM용 모델 준비하기
1. 데이터셋 준비하기  
   CAM은 CNN뒤에 GAP과 소프트맥스 레이어가 붙는다.  
   Grad-CAM은 이런 제약이 없다.  
   CAM은 클래스에 대한 활성화 정도를 나타낸 지도.  
   분류 모델이 기본이다.  
   목표는 이미지 내에 클래스가 활성화된 위치를 찾는 것이므로 위치정보도 필요하다  
   tensorflow datasets에서 Cars196 데이터셋을 사용  
   이 데이터셋에서 라벨에 위치정보인 바운딩 박스 정보를 포함하고 있다.  
   https://www.tensorflow.org/datasets/catalog/cars196  

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import copy
import cv2
from PIL import Image

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# 최초 수행시에는 다운로드가 진행됩니다. 오래 걸릴 수 있으니 유의해 주세요.  
import urllib3
urllib3.disable_warnings()
(ds_train, ds_test), ds_info = tfds.load(
    'cars196',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [ ]:
tfds.show_examples(ds_train, ds_info)

In [ ]:
tfds.show_examples(ds_test, ds_info)

2. 물체의 위치정보  
   bbox를 표시하는 방법들  
   'xywh'와 'minmax'를 주로 사용한다.  
   'xywh'는 바운딩박스의 중심점을 x, y로 표기하고 너비와 높이를 w, h로 표기한다.  
   x, y는 좌측상단 좌표일 수 있다.  
   'minmax'는 바운딩박스 좌표의 최소값과 최대값을 표기한다.(x_min, x_max, y_min, y_max)  
   전체 이미지의 너비와 높이를 기준으로 normalize한 상대적인 값을 표기한 것이다.

In [ ]:
ds_info.features

3. CAM을 위한 모델 만들기  
   이미지넷 데이터로 훈련된 resnet50을 기반으로 하고 pooling layer뒤에 소프트맥스 레이어를 붙인다.

In [ ]:
num_classes = ds_info.features["label"].num_classes
base_model = keras.applications.resnet.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224,3),
    pooling='avg',
)
x = base_model.output
preds = keras.layers.Dense(num_classes, activation='softmax')(x)
cam_model=keras.Model(inputs=base_model.input, outputs=preds)

In [ ]:
cam_model.summary()

4. CAM 모델 학습하기  
   bbox는 CAM모델 학습에 필요하지 않으므로 제외한다.  
   CAM모델은 object detection이나 segmentation에 활용될 수 있지만,  
   바운딩 박스를 직접 사용하지 않고 weakly supervised learning을 통해 물체 영역을 간접적으로 학습시키는 방식이다.

In [ ]:
def normalize_and_resize_img(input):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(input['image'], [224, 224])
    input['image'] = tf.cast(image, tf.float32) / 255.
    return input['image'], input['label']

def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=2
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
ds_train_norm = apply_normalize_on_dataset(ds_train)
ds_test_norm = apply_normalize_on_dataset(ds_test)

In [ ]:
tf.random.set_seed(2020)
cam_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01),
    metrics=['accuracy'],
)

In [ ]:
history_cam_model = cam_model.fit(
    ds_train_norm,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/16),
    validation_steps=int(ds_info.splits['test'].num_examples/16),
    epochs=15,
    validation_data=ds_test_norm,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import os

cam_model_path = os.getenv('HOME')+'/aiffel/class_activation_map/cam_model.h5'
cam_model.save(cam_model_path)

### CAM

In [ ]:
# 한 장을 뽑는 함수

def get_one(ds):
    ds = ds.take(1)
    sample_data = list(ds.as_numpy_iterator())
    bbox = sample_data[0]['bbox']
    image = sample_data[0]['image']
    label = sample_data[0]['label']
    return sample_data[0]

In [ ]:
item = get_one(ds_test)
print(item['label'])
plt.imshow(item['image'])

In [ ]:
cam_model = tf.keras.models.load_model(cam_model_path)

CAM을 만들기 위해서는 특성맵, 클래스 별 확률을 얻기 위한 소프트맥스 레이어의 가중치, 원하는 클래스의 출력값이 필요하다.  
그리고 이미지에서 모델이 어떤 부분을 보는지 시각화하려면 입력 이미지 사이즈와 같게 만들어야 한다.  

In [ ]:
def generate_cam(model, item):
    item = copy.deepcopy(item)
    width = item['image'].shape[1]
    height = item['image'].shape[0]
    
    img_tensor, class_idx = normalize_and_resize_img(item)
    
    # 학습한 모델에서 원하는 Layer의 output을 얻기 위해서 모델의 input과 output을 새롭게 정의해줍니다.
    # model.layers[-3].output에서는 우리가 필요로 하는 GAP 이전 Convolution layer의 output을 얻을 수 있습니다.
    cam_model = tf.keras.models.Model([model.inputs], [model.layers[-3].output, model.output])
    conv_outputs, predictions = cam_model(tf.expand_dims(img_tensor, 0))
    
    conv_outputs = conv_outputs[0, :, :, :]
    class_weights = model.layers[-1].get_weights()[0] #마지막 모델의 weight activation을 가져옵니다.
    
    cam_image = np.zeros(dtype=np.float32, shape=conv_outputs.shape[0:2])
    for i, w in enumerate(class_weights[:, class_idx]):
        # W * f 를 통해 class별 activation map을 계산합니다.
        cam_image += w * conv_outputs[:, :, i]

    cam_image /= np.max(cam_image) # activation score를 normalize합니다.
    cam_image = cam_image.numpy()
    cam_image = cv2.resize(cam_image, (width, height)) # 원래 이미지의 크기로 resize합니다.
    return cam_image

In [ ]:
cam_image = generate_cam(cam_model, item)
plt.imshow(cam_image)

In [ ]:
# CAM 이미지와 원본 이미지를 겹쳐보자.
def visualize_cam_on_image(src1, src2, alpha=0.5):
    beta = (1.0 - alpha)
    merged_image = cv2.addWeighted(src1, alpha, src2, beta, 0.0)
    return merged_image

In [ ]:
origin_image = item['image'].astype(np.uint8)
cam_image_3channel = np.stack([cam_image*255]*3, axis=-1).astype(np.uint8)

blended_image = visualize_cam_on_image(cam_image_3channel, origin_image)
plt.imshow(blended_image)

### Grad-CAM

In [ ]:
item = get_one(ds_test)
print(item['label'])
plt.imshow(item['image'])

In [ ]:
def generate_grad_cam(model, activation_layer, item):
    item = copy.deepcopy(item)
    width = item['image'].shape[1]
    height = item['image'].shape[0]
    img_tensor, class_idx = normalize_and_resize_img(item)
    
    # Grad cam에서도 cam과 같이 특정 레이어의 output을 필요로 하므로 모델의 input과 output을 새롭게 정의합니다.
    # 이때 원하는 레이어가 다를 수 있으니 해당 레이어의 이름으로 찾은 후 output으로 추가합니다.
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(activation_layer).output, model.output])
    
    # Gradient를 얻기 위해 tape를 사용합니다.
    with tf.GradientTape() as tape:
        conv_output, pred = grad_model(tf.expand_dims(img_tensor, 0))
    
        loss = pred[:, class_idx] # 원하는 class(여기서는 정답으로 활용) 예측값을 얻습니다.
        output = conv_output[0] # 원하는 layer의 output을 얻습니다.
        grad_val = tape.gradient(loss, conv_output)[0] # 예측값에 따른 Layer의 gradient를 얻습니다.

    weights = np.mean(grad_val, axis=(0, 1)) # gradient의 GAP으로 class별 weight를 구합니다.
    grad_cam_image = np.zeros(dtype=np.float32, shape=conv_output.shape[0:2])
    for k, w in enumerate(weights):
        # 각 class별 weight와 해당 layer의 output을 곱해 class activation map을 얻습니다.
        grad_cam_image += w * output[:, :, k]
        
    grad_cam_image /= np.max(grad_cam_image)
    grad_cam_image = grad_cam_image.numpy()
    grad_cam_image = cv2.resize(grad_cam_image, (width, height))
    return grad_cam_image

In [ ]:
grad_cam_image = generate_grad_cam(cam_model, 'conv5_block3_out', item)
plt.imshow(grad_cam_image)

In [ ]:
grad_cam_image = generate_grad_cam(cam_model, 'conv4_block3_out', item)
plt.imshow(grad_cam_image)

In [ ]:
grad_cam_image = generate_grad_cam(cam_model, 'conv3_block3_out', item)
plt.imshow(grad_cam_image)

### Detection with CAM
CAM에서 물체의 위치를 찾는 detection해보기

In [ ]:
item = get_one(ds_test)
print(item['label'])
plt.imshow(item['image'])

In [ ]:
cam_image = generate_cam(cam_model, item)
plt.imshow(cam_image)

In [ ]:
# 바운딩 박스를 만드는 함수
# score_thresh: 역치값 이하의 바운딩 박스는 삭제
# OpenCV의 findcontours(), minAreaRect()를 이용하여 사각형 찾기
# rotated_rect라는 회전된 라운딩 박스 얻음.
# boxPoints()을 이용하여 꼭지점으로 바꿔줌.

def get_bbox(cam_image, score_thresh=0.05):
    low_indicies = cam_image <= score_thresh
    cam_image[low_indicies] = 0
    cam_image = (cam_image*255).astype(np.uint8)
    
    contours,_ = cv2.findContours(cam_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    rotated_rect = cv2.minAreaRect(cnt)
    rect = cv2.boxPoints(rotated_rect)
    rect = np.int0(rect)
    return rect

In [ ]:
image = copy.deepcopy(item['image'])
rect = get_bbox(cam_image)
rect

In [ ]:
image = cv2.drawContours(image,[rect],0,(0,0,255),2)
plt.imshow(image)

Intersection Over Union  
모델에 의해 도출된 바운딩 박스와 정답 데이터의 바운딩 박스를 비교하기  
IoU: 두 개 영역의 합집합인 Union으로 교집합 영역인 intersection으로 나누어준 값  
https://www.youtube.com/watch?v=ANIzQ5G-XPE

In [ ]:
# rect의 좌표는 (x, y) 형태로, bbox는 (y_min, x_min, y_max, x_max)의 normalized 형태로 주어집니다. 
def rect_to_minmax(rect, image):
    bbox = [
        rect[:,1].min()/float(image.shape[0]),  #bounding box의 y_min
        rect[:,0].min()/float(image.shape[1]),  #bounding box의 x_min
        rect[:,1].max()/float(image.shape[0]),  #bounding box의 y_max
        rect[:,0].max()/float(image.shape[1])   #bounding box의 x_max
    ]
    return bbox

In [ ]:
pred_bbox = rect_to_minmax(rect, item['image'])
pred_bbox

In [ ]:
# ground truth bbox
item['bbox']

In [ ]:
def get_iou(boxA, boxB):
    y_min = max(boxA[0], boxB[0])
    x_min= max(boxA[1], boxB[1])
    y_max = min(boxA[2], boxB[2])
    x_max = min(boxA[3], boxB[3])
    
    interArea = max(0, x_max - x_min) * max(0, y_max - y_min)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [ ]:
get_iou(pred_bbox, item['bbox'])